# Using Fairness Dashboard with Loan Allocation Data

This notebook shows how to use [`Fairlearn`](https://fairlearn.github.io/) and the Responsible AI Widget's Fairness dashboard to generate predictors for the Census dataset. This dataset is a classification problem - given a range of data about 32,000 individuals, predict whether their annual income is above or below fifty thousand dollars per year.

For the purposes of this notebook, we shall treat this as a loan decision problem. We will pretend that the label indicates whether or not each individual repaid a loan in the past. We will use the data to train a predictor to predict whether previously unseen individuals will repay a loan or not. The assumption is that the model predictions are used to decide whether an individual should be offered a loan.

We will first train a fairness-unaware predictor and use the fairness dashboard to demonstrate how this model leads to unfair decisions (under a specific notion of fairness called *demographic parity*). We then mitigate unfairness by applying the `GridSearch` algorithm from `Fairlearn` package.

## Install Required Packages

In [3]:
# %pip install --upgrade fairlearn
# %pip install --upgrade raiwidgets

# %pip install -e "/mnt/c/Users/t-alexquach/Desktop/fairlearn"
# %pip install raiwidgets

# %pip install -e "/mnt/c/Users/t-alexquach/Music/fairlearn"
%pip install -e "/mnt/c/Users/t-alexquach/Music/responsible-ai-widgets/raiwidgets"
# %pip install -e "/mnt/c/Users/t-alexquach/Music/responsible-ai-widgets/responsibleai"

Obtaining file:///mnt/c/Users/t-alexquach/Music/responsible-ai-widgets/raiwidgets
  Running setup.py develop for raiwidgets
Note: you may need to restart the kernel to use updated packages.


## Load and preprocess the data set

For simplicity, we import the data set from the `shap` package, which contains the data in a cleaned format. We start by importing the various modules we're going to use:

In [1]:
from raiwidgets import FairnessDashboard

from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity
from fairlearn.metrics import MetricFrame, selection_rate

from sklearn import svm, neighbors, tree
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_openml

import pandas as pd

import numpy as np

ModuleNotFoundError: No module named 'raiwidgets._input_processing'

We can now load and inspect the data from `openml`:

In [ ]:
data = fetch_openml(data_id=1590, as_frame=True)
X_raw = data.data
y_true = (data.target == '>50K') * 1

X_raw["race"].value_counts().to_dict()

We are going to treat the sex of each individual as a protected attribute (where 0 indicates female and 1 indicates male), and in this particular case we are going separate this attribute out and drop it from the main data. We then perform some standard data preprocessing steps to convert the data into a format suitable for the ML algorithms

In [2]:
sensitive_features = X_raw[['sex','race']]
X = X_raw.drop(labels=['sex', 'race'],axis = 1)
X = pd.get_dummies(X)

sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

le = LabelEncoder()
y_true = le.fit_transform(y_true)

NameError: name 'X_raw' is not defined

Finally, we split the data into training and test sets:

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, sensitive_features_train, sensitive_features_test = \
    train_test_split(X_scaled, y_true, sensitive_features,
                     test_size = 0.2, random_state=0, stratify=y_true)

# Work around indexing bug
X_train = X_train.reset_index(drop=True)
sensitive_features_train = sensitive_features_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
sensitive_features_test = sensitive_features_test.reset_index(drop=True)

NameError: name 'X_scaled' is not defined

## Train a fairness-unaware predictor

To show the effect of `Fairlearn` we will first train a standard ML predictor that does not incorporate fairness For speed of demonstration, we use a simple logistic regression estimator from `sklearn`:

In [ ]:
unmitigated_predictor = LogisticRegression(solver='liblinear', fit_intercept=True)

unmitigated_predictor.fit(X_train, y_train)

We can load this predictor into the Fairness dashboard, and examine how it is unfair (there is a warning about AzureML since we are not yet integrated with that product):

In [ ]:
y_pred = unmitigated_predictor.predict(X_test)

FairnessDashboard(sensitive_features=sensitive_features_test,
                  y_true=y_test,
                  y_pred=y_pred)

Looking at the disparity in accuracy, we see that males have an error rate about three times greater than the females. More interesting is the disparity in opportunitiy - males are offered loans at three times the rate of females.

Despite the fact that we removed the feature from the training data, our predictor still discriminates based on sex. This demonstrates that simply ignoring a protected attribute when fitting a predictor rarely eliminates unfairness. There will generally be enough other features correlated with the removed attribute to lead to disparate impact.

## Mitigation with GridSearch

The `GridSearch` class in `Fairlearn` implements a simplified version of the exponentiated gradient reduction of [Agarwal et al. 2018](https://arxiv.org/abs/1803.02453). The user supplies a standard ML estimator, which is treated as a blackbox. `GridSearch` works by generating a sequence of relabellings and reweightings, and trains a predictor for each.

For this example, we specify demographic parity (on the protected attribute of sex) as the fairness metric. Demographic parity requires that individuals are offered the opportunity (are approved for a loan in this example) independent of membership in the protected class (i.e., females and males should be offered loans at the same rate). We are using this metric for the sake of simplicity; in general, the appropriate fairness metric will not be obvious.

In [ ]:
sweep = GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                   constraints=DemographicParity(),
                   grid_size=70)

Our algorithms provide `fit()` and `predict()` methods, so they behave in a similar manner to other ML packages in Python. We do however have to specify two extra arguments to `fit()` - the column of protected attribute labels, and also the number of predictors to generate in our sweep.

After `fit()` completes, we extract the full set of predictors from the `GridSearch` object.

In [ ]:
sweep.fit(X_train, y_train,
          sensitive_features=sensitive_features_train.sex)

predictors = sweep.predictors_

We could load these predictors into the Fairness dashboard now. However, the plot would be somewhat confusing due to their number. In this case, we are going to remove the predictors which are dominated in the accuracy-disparity space by others from the sweep (note that the disparity will only be calculated for the protected attribute; other potentially protected attributes will not be mitigated). In general, one might not want to do this, since there may be other considerations beyond the strict optimization of accuracy and disparity (of the given protected attribute).

In [ ]:
accuracies, disparities = [], []

for predictor in predictors:
    accuracy_metric_frame = MetricFrame(accuracy_score, y_train, predictor.predict(X_train), sensitive_features=sensitive_features_train.sex)
    selection_rate_metric_frame = MetricFrame(selection_rate, y_train, predictor.predict(X_train), sensitive_features=sensitive_features_train.sex)
    accuracies.append(accuracy_metric_frame.overall)
    disparities.append(selection_rate_metric_frame.difference())
    
all_results = pd.DataFrame( {"predictor": predictors, "accuracy": accuracies, "disparity": disparities})

all_models_dict = {"unmitigated": unmitigated_predictor}
dominant_models_dict = {"unmitigated": unmitigated_predictor}
base_name_format = "grid_{0}"
row_id = 0
for row in all_results.itertuples():
    model_name = base_name_format.format(row_id)
    all_models_dict[model_name] = row.predictor
    accuracy_for_lower_or_eq_disparity = all_results["accuracy"][all_results["disparity"] <= row.disparity]
    if row.accuracy >= accuracy_for_lower_or_eq_disparity.max():
        dominant_models_dict[model_name] = row.predictor
    row_id = row_id + 1

We can construct predictions for all the models, and also for the dominant models:

In [ ]:
dashboard_all = {}
for name, predictor in all_models_dict.items():
    value = predictor.predict(X_test)
    dashboard_all[name] = value
    
dominant_all = {}
for name, predictor in dominant_models_dict.items():
    dominant_all[name] = predictor.predict(X_test)

FairnessDashboard(sensitive_features=sensitive_features_test, 
                  y_true=y_test.tolist(),
                  y_pred=dominant_all)

We see a Pareto front forming - the set of predictors which represent optimal tradeoffs between accuracy and disparity in predictions. In the ideal case, we would have a predictor at (1,0) - perfectly accurate and without any unfairness under demographic parity (with respect to the protected attribute "sex"). The Pareto front represents the closest we can come to this ideal based on our data and choice of estimator. Note the range of the axes - the disparity axis covers more values than the accuracy, so we can reduce disparity substantially for a small loss in accuracy.

By clicking on individual models on the plot, we can inspect their metrics for disparity and accuracy in greater detail. In a real example, we would then pick the model which represented the best trade-off between accuracy and disparity given the relevant business constraints.